In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [2]:
owners, n_owners = dr.getOwners()
n_tracks = dr.n_tracks

In [3]:
train = dr.train.T
d = {}
for row in train:
    p = row[0]
    t = row[1]
    if p in d:
        d[p].append(t)
    else:
        d[p] = []
        d[p].append(t)

In [4]:
xxx = []
for row in owners.T:
    o = row[1]
    p = row[0]
    if p not in d:
        continue #we don't have this playlist in train
    t_p = d[p]
    for t in t_p:
        xxx.append([o,t])
xxx = np.array(xxx)

In [5]:
#build xxx matrix

r,c,v = [],[],[]
for row in xxx:
    r.append(row[0])
    c.append(row[1])
    v.append(1)

xcm = sp.coo_matrix((v, (r, c)), shape=(n_owners, n_tracks ),dtype=np.float).tocsr().T
xcm.data = np.ones(len(xcm.data))

In [3]:
model = BM25Recommender(K=100)
model.fit(a)
s = model.similarity
r_x = urm*s.T
r_x = normalize(r_x, axis=1)
if evaluation: print dr.evaluateMAP(r_x,verbose=False)
r_x = dr.reduceRM(r_x)

0.0504691944444


In [2]:
a = dr.getOwnersICM_csr(ones=True)